In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [0]:
fato_envios_final = spark.table("grao.silver.fato_envios")
dim_localizacao = spark.table("grao.silver.dim_localizacao")
dim_metodo_envio = spark.table("grao.silver.dim_metodo_envio")
df_calendario = spark.table("grao.bronze.calendario") 

In [0]:
df_calendario_com_id = (
    spark.table("grao.bronze.calendario")
    .withColumn("id_data", F.row_number().over(Window.orderBy("data")))
)

In [0]:
print("\n--- Criando Tabela Gold 3: Gold_Resumo_Mensal_Operacoes ---")

gold_resumo_mensal = (
    fato_envios_final 
    .join(
        df_calendario_com_id, 
        fato_envios_final["id_data_envio"] == df_calendario_com_id["id_data"], 
        "left"
    )
    .groupBy("ano", "mes") 
    .agg(
        F.sum("preco").alias("receita_total_mensal"),
        F.count("id_envio").alias("total_envios_mensal"),
        F.round((F.sum("peso_g") / 1000), 2).alias("peso_total_transportado_kg")
    )
    .orderBy("ano", "mes")
)


gold_resumo_mensal.write.mode("overwrite").format("delta").saveAsTable("grao.gold.resumo_mensal_operacoes")
print("Tabela Gold 'resumo_mensal_operacoes' criada com sucesso.")